## Load available keys

In [ ]:
import ooiod

In [ ]:
key_file = '/home/jovyan/repos/ooi-opendata/secrets/daxsoule.pickle'
keys = ooiod.secrets.load_keys(key_file)

## Download a NetCDF file from OOI results server

In [ ]:
#file1
netcdf_url = 'https://opendap.oceanobservatories.org/async_results/dax.soule@qc.cuny.edu/20191227T065318736Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_20141005T000000-20141005T235959.nc'
netcdf_filename = netcdf_url.split('/')[-1]

In [ ]:
#file2
netcdf_url2 = 'https://opendap.oceanobservatories.org/async_results/dax.soule@qc.cuny.edu/20191228T052214778Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_20191005T000000-20191005T235959.950000.nc'
netcdf_filename = netcdf_url2.split('/')[-1]

In [ ]:
import wget

In [ ]:
wget.download(netcdf_url2);

## Write the original NetCDF file into Azure blob

In [ ]:
from azure.storage.blob import BlockBlobService

In [ ]:
if 'botpt' in keys:
    botpt_service = BlockBlobService('ooiopendata', sas_token = keys['botpt'])
else:
    raise Exception('Botpt container key not available')

In [ ]:
botpt_service.create_blob_from_path('botpt', netcdf_filename, netcdf_filename, validate_content=True);

In [ ]:
for blob in botpt_service.list_blobs('botpt'):
    print(blob.name)

## Load the file into Xarray and save to ooiopendata

In [ ]:
import xarray as xr
import zarr
import logging
logging.getLogger("azure.storage").setLevel(logging.CRITICAL)

In [ ]:
ds = xr.open_dataset(netcdf_filename)
ds = ds.swap_dims({'obs': 'time'})
ds = ds.drop(['deployment','id', 'date_time_string', 'ingestion_timestamp', 'internal_timestamp','port_timestamp',
             'driver_timestamp', 'preferred_timestamp', 'provenance', 'time_sync_flag', 'bottom_pressure_qc_executed',
             'bottom_pressure_qc_results'])
del ds['bottom_pressure'].attrs['coordinates']
del ds['press_trans_temp'].attrs['coordinates']
del ds['sensor_id'].attrs['coordinates']
ds

### Get a list of files from the server

In [ ]:
%%time
blob_urls = ooiod.blobs.list_blobs('ooiopendata', 'botpt')

In [ ]:
blob_urls

### Get file sizes

In [ ]:
import dask
from dask import delayed

In [ ]:
from dask.distributed import Client

client = Client("tcp://10.0.128.32:43341")
client

### Create a Pandas dataframe from these Lists

In [ ]:
import pandas as pd

In [ ]:
dbbotpt = pd.DataFrame(
    {'blob_url': blob_urls,
    })
dbbotpt.tail()

In [ ]:
ds1 = xr.open_dataset('https://ooiopendata.blob.core.windows.net/botpt/NOAA_NCDC_ERSST_v3b_SST.nc')
ds1

### Create new NetCDF file

In [ ]:
out = netcdf_url2.split('/')[-1][:-3] + '_edit' + '.nc'
ds.to_netcdf(out)
botpt_service.create_blob_from_path('botpt', out, out, validate_content=True);

### Explore the other links and services provided

### Investigate Intake 

In [ ]:
import intake
intake.output_notebook()

In [ ]:
botpt_da = xr.open_dataset('https://opendap.oceanobservatories.org/thredds/dodsC/ooi/dax.soule@qc.cuny.edu/20191228T052214778Z-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/deployment0001_RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_20191005T000000-20191005T235959.950000.nc' + '#fillmismatch')
botpt_da

In [ ]:
ds.source

In [ ]:
'%s-%s-%s' % (ds.subsite,ds.node,ds.sensor)

In [ ]:
ds['press_trans_temp']

In [ ]:
import textwrap
textwrap.wrap(ds['press_trans_temp'].comment)

In [ ]:
ds['bottom_pressure'].dims

In [ ]:
import matplotlib.pyplot as plt
fig, (ax1,ax2) = plt.subplots(2,1, sharex=True, figsize=(14,9))
ds['bottom_pressure'].plot(ax=ax1,linestyle='None',marker='.',markersize=1)
ds['press_trans_temp'].plot(ax=ax2,linestyle='None',marker='.',markersize=1)